In [1]:
import re
import string
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import xlrd  
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier

Using TensorFlow backend.


In [2]:
p_stemmer = PorterStemmer()
# text preprocession
def split_word(text):
    
    tokens = word_tokenize(text,engine='newmm')

    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,low_window,up_window,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    

def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat,l,u):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+str(l)+","+str(u)+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+str(l)+","+str(u)+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [22]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
    
def train_and_label_context(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType,l,u):
    model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    model.fit(train_X, train_y)
    
    # eval
    predictions_test = model.predict_proba(test_X)
    predictions_train = model.predict_proba(train_X)
    
#     for th in numpy.arange(0.1,1.1,0.1):
#         predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
#         predictions_class_Test = (predictions_test[:,1] >= th).astype('int')
    predictions_class_Train = (predictions_train[:,1] >= 0.3).astype('int')
    predictions_class_Test = (predictions_test[:,1] >= 0.3).astype('int')

    report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,0.3, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0],l,u)

    # add new data

    predictions_unlabeled = model.predict_proba(u_X)
    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > u).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < l).reshape(-1)
    return certain_positives, certain_negatives

In [23]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
    
def train_and_label_content(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType,l,u):
    if(type_class == 'dirty'):
        model = LinearSVC()
    model.fit(train_X, train_y)
    
    # eval
    if(type_class != 'dirty'):
        predictions_test = model.predict_proba(test_X)
        predictions_train = model.predict_proba(train_X)
        predictions_unlabeled = model.predict_proba(u_X)
    
    elif(type_class == 'dirty'):
        predictions_test = model._predict_proba_lr(test_X)
        predictions_train = model._predict_proba_lr(train_X)
        predictions_unlabeled = model._predict_proba_lr(u_X)
    
#     for th in numpy.arange(0.1,1.1,0.1):
#         predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
#         predictions_class_Test = (predictions_test[:,1] >= th).astype('int')

#         report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0],l,u)

#     predictions_class_Train = (predictions_train[:,1] >= 0.4).astype('int')
#     predictions_class_Test = (predictions_test[:,1] >= 0.4).astype('int')

#     report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,0.4, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0],l,u)

    # add new data

    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > u).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < l).reshape(-1)
    return certain_positives, certain_negatives

In [7]:
dtypes = {
    'comment_text':'str',
    'comment_id':'str',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str'
}
#,
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count']
#comment_author
content_features = ['comment_text']
labels = ['rude','figurative','offensive','dirty']


reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [8]:
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non.csv', dtype=dtypes)
Unlabeled_Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/unlabeled_data_temp1.csv', dtype=dtypes)

#remove duplicate
label_id_set = set(Corpus['comment_id'])
unlabel_id_set = set(Unlabeled_Corpus['comment_id'])
duplicate_comment_id_set = label_id_set.intersection(unlabel_id_set)
Unlabeled_Corpus = Unlabeled_Corpus.loc[~Unlabeled_Corpus['comment_id'].isin(duplicate_comment_id_set)]
Unlabeled_Corpus.reset_index(inplace=True)

In [9]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)
Unlabeled_Corpus[reaction_columns] = Unlabeled_Corpus[reaction_columns].replace(reaction_map)

In [10]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(
    Corpus[content_features + context_features], 
    Corpus[labels], test_size=0.2, random_state=1000);

#Tokenizaiton
Tfidf_vect = TfidfVectorizer(tokenizer=split_word, min_df=2, max_features=10000, token_pattern=None)

#fit only train, unlabel
Tfidf_vect.fit(pd.concat([Train_X['comment_text'],Unlabeled_Corpus['comment_text']]))

#transform train, test, unlabel
Train_X_Tfidf = Tfidf_vect.transform(Train_X['comment_text'])
Test_X_Tfidf = Tfidf_vect.transform(Test_X['comment_text'])
Unlabeled_X_Tfidf = Tfidf_vect.transform(Unlabeled_Corpus['comment_text'])

In [11]:
from scipy.sparse import hstack, csr_matrix, vstack
train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
unlabeled_feat_context = Unlabeled_Corpus[context_features].values

train_feat_content = Train_X_Tfidf
test_feat_content = Test_X_Tfidf
unlabeled_feat_content = Unlabeled_X_Tfidf

In [12]:
low_window = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

In [13]:
up_window = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

In [20]:
label = 'dirty'
file = 'test_context_window_dirty.csv'
createGeneralEvaluation(file)
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
time = 1
for i,l in enumerate(low_window):
    for j, u in enumerate(up_window):
        train_content = train_feat_content
        content_train_y = _train_y
        context_train_y = _train_y

        context_certain_set = set()
        context_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
        context_certain_labels[label] = -1

        # certain data from context
        _xp, _xn = train_and_label_context(
        train_feat_context, context_train_y, 
        test_feat_context, _test_y,
        unlabeled_feat_context, file, label, time, "context",l,u)

        # Update certain data from context to content model
        context_certain_set.update(list(_xp))
        context_certain_set.update(list(_xn))
        context_certain_labels.loc[_xp, label] = 1
        context_certain_labels.loc[_xn, label] = 0

        _xxl = list(context_certain_set)
        _xx_X = unlabeled_feat_content[_xxl]
        _xx_y = context_certain_labels.loc[_xxl, label]

        train_content = vstack([train_content, _xx_X])
        content_train_y = np.concatenate([content_train_y, _xx_y.values], axis=0)

        _cp, _cn = train_and_label_content(
        train_content, content_train_y, 
        test_feat_content, _test_y,
        unlabeled_feat_content, file, label, time, "content",l,u)


        

Test data of content
dirty Accuracy score: 97.63346175013758
Confusion Matrix: 
 [[3477   32]
 [  54   71]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      3509
         1.0       0.69      0.57      0.62       125

    accuracy                           0.98      3634
   macro avg       0.84      0.78      0.81      3634
weighted avg       0.97      0.98      0.98      3634

Area under the curve ROC:  0.9549068110572813
Area under the curve PR:  0.6671249203758256
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 99.72058446552167
Confusion Matrix: 
 [[38453    87]
 [   22   448]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     38540
         1.0       0.84      0.95      0.89       470

    accuracy                           1.00     39010
   macro avg       0.92      0.98      0.95     39010
weighted avg  

Test data of content
dirty Accuracy score: 97.6609796367639
Confusion Matrix: 
 [[3481   28]
 [  57   68]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      3509
         1.0       0.71      0.54      0.62       125

    accuracy                           0.98      3634
   macro avg       0.85      0.77      0.80      3634
weighted avg       0.97      0.98      0.98      3634

Area under the curve ROC:  0.9553650612710174
Area under the curve PR:  0.6652697155101112
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 99.81393800959052
Confusion Matrix: 
 [[65549    88]
 [   35   435]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     65637
         1.0       0.83      0.93      0.88       470

    accuracy                           1.00     66107
   macro avg       0.92      0.96      0.94     66107
weighted avg   

Area under the curve PR:  0.9077191064578191
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 97.57842597688497
Confusion Matrix: 
 [[3491   18]
 [  70   55]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      3509
         1.0       0.75      0.44      0.56       125

    accuracy                           0.98      3634
   macro avg       0.87      0.72      0.77      3634
weighted avg       0.97      0.98      0.97      3634

Area under the curve ROC:  0.9553582217155884
Area under the curve PR:  0.6521579900372394
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 99.83625050034568
Confusion Matrix: 
 [[81882    91]
 [   44   426]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     81973
         1.0       0.82

PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    101049
         1.0       0.84      0.89      0.86       470

    accuracy                           1.00    101519
   macro avg       0.92      0.94      0.93    101519
weighted avg       1.00      1.00      1.00    101519

Area under the curve ROC:  0.9992255811010584
Area under the curve PR:  0.8882785442036817
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 97.49587231700605
Confusion Matrix: 
 [[3493   16]
 [  75   50]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3509
         1.0       0.76      0.40      0.52       125

    accuracy                           0.97      3634
   macro avg       0.87      0.70      0.76      3634
weighted avg       0.97      0.97      0.97      3634

Area under the curve ROC

PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    103211
         1.0       0.84      0.89      0.87       470

    accuracy                           1.00    103681
   macro avg       0.92      0.95      0.93    103681
weighted avg       1.00      1.00      1.00    103681

Area under the curve ROC:  0.9992619539769492
Area under the curve PR:  0.8940205521436738
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 97.52339020363236
Confusion Matrix: 
 [[3492   17]
 [  73   52]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3509
         1.0       0.75      0.42      0.54       125

    accuracy                           0.98      3634
   macro avg       0.87      0.71      0.76      3634
weighted avg       0.97      0.98      0.97      3634

Area under the curve ROC

PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    107994
         1.0       0.84      0.88      0.86       470

    accuracy                           1.00    108464
   macro avg       0.92      0.94      0.93    108464
weighted avg       1.00      1.00      1.00    108464

Area under the curve ROC:  0.9992094024924159
Area under the curve PR:  0.8857022055955549
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 97.52339020363236
Confusion Matrix: 
 [[3493   16]
 [  74   51]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3509
         1.0       0.76      0.41      0.53       125

    accuracy                           0.98      3634
   macro avg       0.87      0.70      0.76      3634
weighted avg       0.97      0.98      0.97      3634

Area under the curve ROC

In [24]:
label = 'dirty'
file = 'test_content_window_dirty.csv'
createGeneralEvaluation(file)
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
time = 1
for i,l in enumerate(low_window):
    for j, u in enumerate(up_window):
        train_context = train_feat_context
        content_train_y = _train_y
        context_train_y = _train_y

        content_certain_set = set()
        content_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
        content_certain_labels[label] = -1

        # certain data from content
        _xp, _xn = train_and_label_content(
        train_feat_content, content_train_y, 
        test_feat_content, _test_y,
        unlabeled_feat_content, file, label, time, "content",l,u)

        # Update certain data from context to content model
        content_certain_set.update(list(_xp))
        content_certain_set.update(list(_xn))
        content_certain_labels.loc[_xp, label] = 1
        content_certain_labels.loc[_xn, label] = 0

        _xxl = list(content_certain_set)
        _xx_X = unlabeled_feat_context[_xxl]
        _xx_y = content_certain_labels.loc[_xxl, label]

        train_context = vstack([train_context, _xx_X])
        context_train_y = np.concatenate([context_train_y, _xx_y.values], axis=0)

        _cp, _cn = train_and_label_context(
        train_context, context_train_y, 
        test_feat_context, _test_y,
        unlabeled_feat_context, file, label, time, "context",l,u)


        

Test data of context
dirty Accuracy score: 96.0924600990644
Confusion Matrix: 
 [[3423   86]
 [  56   69]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      3509
         1.0       0.45      0.55      0.49       125

    accuracy                           0.96      3634
   macro avg       0.71      0.76      0.74      3634
weighted avg       0.97      0.96      0.96      3634

Area under the curve ROC:  0.9274904531205472
Area under the curve PR:  0.4853575587798872
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 99.85602632661457
Confusion Matrix: 
 [[14044    21]
 [    0   521]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       0.96      1.00      0.98       521

    accuracy                           1.00     14586
   macro avg       0.98      1.00      0.99     14586
weighted avg   

Test data of context
dirty Accuracy score: 95.92735277930655
Confusion Matrix: 
 [[3421   88]
 [  60   65]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      3509
         1.0       0.42      0.52      0.47       125

    accuracy                           0.96      3634
   macro avg       0.70      0.75      0.72      3634
weighted avg       0.96      0.96      0.96      3634

Area under the curve ROC:  0.9343858649187803
Area under the curve PR:  0.48975285873045027
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 99.80290879434553
Confusion Matrix: 
 [[14190    29]
 [    0   495]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14219
         1.0       0.94      1.00      0.97       495

    accuracy                           1.00     14714
   macro avg       0.97      1.00      0.99     14714
weighted avg 

Test data of context
dirty Accuracy score: 96.23004953219592
Confusion Matrix: 
 [[3439   70]
 [  67   58]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      3509
         1.0       0.45      0.46      0.46       125

    accuracy                           0.96      3634
   macro avg       0.72      0.72      0.72      3634
weighted avg       0.96      0.96      0.96      3634

Area under the curve ROC:  0.9222992305500143
Area under the curve PR:  0.4882580908844225
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 99.88359847015133
Confusion Matrix: 
 [[17536    21]
 [    0   484]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     17557
         1.0       0.96      1.00      0.98       484

    accuracy                           1.00     18041
   macro avg       0.98      1.00      0.99     18041
weighted avg  

Test data of context
dirty Accuracy score: 96.03742432581178
Confusion Matrix: 
 [[3435   74]
 [  70   55]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      3509
         1.0       0.43      0.44      0.43       125

    accuracy                           0.96      3634
   macro avg       0.70      0.71      0.71      3634
weighted avg       0.96      0.96      0.96      3634

Area under the curve ROC:  0.8885049871758335
Area under the curve PR:  0.45691790738881644
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 99.92703301127214
Confusion Matrix: 
 [[39244    29]
 [    0   471]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     39273
         1.0       0.94      1.00      0.97       471

    accuracy                           1.00     39744
   macro avg       0.97      1.00      0.98     39744
weighted avg 

Area under the curve ROC:  0.9999978065176021
Area under the curve PR:  0.9996051882166852
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 95.87231700605393
Confusion Matrix: 
 [[3435   74]
 [  76   49]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      3509
         1.0       0.40      0.39      0.40       125

    accuracy                           0.96      3634
   macro avg       0.69      0.69      0.69      3634
weighted avg       0.96      0.96      0.96      3634

Area under the curve ROC:  0.8869090909090909
Area under the curve PR:  0.42059513705740925
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 99.96516973946964
Confusion Matrix: 
 [[85632    30]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1

PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    102184
         1.0       0.94      1.00      0.97       470

    accuracy                           1.00    102654
   macro avg       0.97      1.00      0.98    102654
weighted avg       1.00      1.00      1.00    102654

Area under the curve ROC:  0.9999990734278256
Area under the curve PR:  0.9997992520775545
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 96.28508530544853
Confusion Matrix: 
 [[3449   60]
 [  75   50]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      3509
         1.0       0.45      0.40      0.43       125

    accuracy                           0.96      3634
   macro avg       0.72      0.69      0.70      3634
weighted avg       0.96      0.96      0.96      3634

Area under the curve ROC